<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>DRAFT: Coadd Recreation</b> <br>
Contact author(s): Melissa Graham <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: intermediate <br>

In [ ]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** Recreate a Coadded image from a subset of the input calexps.

**Skills:** Use of pipetasks for image coaddition. Creating and writing to Butler collections. Properties of deepCoadds.

**LSST Data Products:** deepCoadd, calexp

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:** Originally developed by Melissa Graham and Clare Saunders.
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

This notebook shows how to retrieve information about the individual images that contributed to a deepCoadd, and how to make a new Coadded image using only a subset of the inputs.

In the past you might have used iraf's imcombine, or Astromatics's SWarp, to coadd images.
This notebook demonstrates the appropriate methods for coadding LSST images with the LSST Science Pipelines.

Science applications of coadding a subsets of LSST images includes searching for faint, slowly-evolving transients or variables (e.g.,, coadding images by season), using the effects of differential chromatic refraction (e.g., coadding images in bins of airmass), or perhaps searching for low surface brightness features (e.g., coadding only dark-time images with the faintest sky backgrounds).

### 1.1 Package Imports

_Provide explanation or external links to package documentation, where appropriate._

In [ ]:
# standard python packages for numerical processing, plotting, and databases
import time
import numpy as np

import matplotlib
font = {'size' : 14}
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt

import pandas
pandas.set_option('display.max_rows', 1000)

# astropy package for time unit conversions
from astropy.time import Time

# lsst packages for data access and display
import lsst.daf.butler as dafButler
import lsst.geom
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

# lsst packages for executing pipeline tasks
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline, TaskDef, Instrument
# from lsst.daf.butler import Butler, DatasetType, CollectionType

### 1.2 Set Up

In [ ]:
repo = 'dp02'
collection = '2.2i/runs/DP0.2/v23_0_2/PREOPS-905/step_all'

Known issue: it is OK to ignore a pink-window message saying "WARNING: version mismatch between CFITSIO header (v4.000999999999999) and linked library (v4.01)."

In [ ]:
butler = dafButler.Butler(repo, collections=collection)

## 2. Identify the calexps to combine

### 2.1. Start with a deepCoadd

Identify and retrieve the deepCoadd to be recreated.

For this example, the coordinates of a known galaxy cluster are used.

It takes 4-5 seconds to retrieve a single deepCoadd from the butler.

In [ ]:
%%time

my_ra_deg = 55.745834
my_dec_deg = -32.269167

my_spherePoint = lsst.geom.SpherePoint(my_ra_deg*lsst.geom.degrees,
                                       my_dec_deg*lsst.geom.degrees)

skymap = butler.get('skyMap')
my_tract = skymap.findTract(my_spherePoint)
my_patch = my_tract.findPatch(my_spherePoint)

my_dataId = {'band': 'i', 'tract': my_tract.tract_id, 
             'patch': my_patch.getSequentialIndex()}
my_deepCoadd = butler.get('deepCoadd', dataId=my_dataId)

# clean up
del my_ra_deg, my_dec_deg, my_spherePoint, my_tract, my_patch

Uncomment the following cell to show the deepCoadd image.

In [ ]:
# fig = plt.figure(figsize=(6, 4))
# afw_display = afwDisplay.Display(1)
# afw_display.scale('asinh', 'zscale')
# afw_display.mtv(my_deepCoadd.image)
# plt.gca().axis('off')

It is not necessary to know the bounding box for a deepCoadd in order to find all of the calexps that were used to assemble it.
But, if you want to learn more about the deepCoadd metadata, such as bounding box, corners, and the World Coordinate System (WCS), uncomment and execute the following cells.

In [ ]:
# my_deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=my_dataId)
# print('bbox')
# print(my_deepCoadd_bbox.beginX, my_deepCoadd_bbox.beginY, 
#       my_deepCoadd_bbox.endX, my_deepCoadd_bbox.endY)

# print('')
# print('corners')
# print(my_deepCoadd_bbox.getCorners())

# print('')
# print('wcs')
# my_deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=my_dataId)
# print(my_deepCoadd_wcs)

# # clean up
# del my_deepCoadd_bbox, my_deepCoadd_wcs

### 2.2. Retrieve the deepCoadd's input visits

It takes 2-3 seconds to retrieve the coadd inputs from the butler.

In [ ]:
%%time

my_coadd_inputs = butler.get("deepCoadd_calexp.coaddInputs", my_dataId)

The coadd inputs table of visits can be displayed as an astropy table, if you want to view the contents.

In [ ]:
# my_coadd_inputs.visits.asAstropy()

The length of this table, 161, indicates that 161 separate visits contributed to this deepCoadd.

In [ ]:
len(my_coadd_inputs.visits)

It is not necessary to make a list of all deepCoadd input visit ids, but if you wanted to, you could do it and display it like this.

In [ ]:
# my_coadd_visits = my_coadd_inputs.visits['id']
# my_coadd_visits

### 2.3. Get metadata for the input visits

Retrieve the modified julian date (MJD) of the input visits from the visitTable.

First, get the entire visit table.

In [ ]:
visitTableRef = list(butler.registry.queryDatasets('visitTable'))

In [ ]:
# visitTableRef

In [ ]:
visitTable = butler.get(visitTableRef[0])

In [ ]:
# visitTable

The fact that the id column for both the my_coadd_inputs.vists table and the visitTable is the visit number (visit id) makes it simple to retrieve the MJDs of our coadd input visits.

In [ ]:
my_coadd_visits_mjds = visitTable.loc[my_coadd_inputs.visits['id']]['expMidptMJD']

These list of MJDs have 161 elements, for the 161 separate visits contributed to this deepCoadd.

In [ ]:
len(my_coadd_visits_mjds)

### 2.4. Identify input visits to recreate Coadd

Identify input visits with MJD between 60925 abd 60955.

In [ ]:
range_start = 60925
range_end = 60955

fig, ax = plt.subplots( 2, figsize=(10,10) )

ax[0].hist(my_coadd_visits_mjds, bins=150, color='dodgerblue')
ax[0].set_xlabel('MJD')
ax[0].set_ylabel('Number of Visits')
ax[0].axvline(range_start, ls='dashed', color='darkorange')
ax[0].axvline(range_end, ls='dashed', color='darkorange')

ax[1].hist(my_coadd_visits_mjds, bins=150, color='dodgerblue')
ax[1].set_xlabel('MJD')
ax[1].set_ylabel('Number of Visits')
ax[1].set_xlim([60880,60985])
ax[1].axvline(range_start, ls='dashed', color='darkorange')
ax[1].axvline(range_end, ls='dashed', color='darkorange')
ax[1].text(range_start+1, 7.5, 'date range', color='darkorange')
ax[1].text(range_start+1, 7.0, 'of interest', color='darkorange')

plt.show()

From the lower plot above, there are 6 visits in the data range of interest.

Put this list of visits into a string, formatted as a tuple (within round brackets and separated by commas) for use in a query below.

In [ ]:
my_range = np.array((my_coadd_visits_mjds > range_start) & 
                    (my_coadd_visits_mjds < range_end))

my_visits = my_coadd_inputs.visits[my_range]

my_visits_tupleString = "("+",".join(my_visits['id'].astype(str))+")"
print(my_visits_tupleString)

# visitString = f"visit in ({tupleString})"
# queryString = f"patch = {my_patch} AND {visitString} AND skymap = 'skymap'"

### 2.5. Exercise for the learner

Use airmass constraints instead of MJD to identify the subset of visits to coadd.

> Hint: start with 
> `my_coadd_visits_airmass = visitTable.loc[my_coadd_inputs.visits['id']]['airmass']`

## 3. Create a coadd with the subset of visits 

<br>
<br>
<br>


Work in progress. Trying two approaches.

<br>
<br>

### 3.1. Try using the pipe task assembleCoadd

https://pipelines.lsst.io/modules/lsst.pipe.tasks/index.html#lsst-pipe-tasks

https://pipelines.lsst.io/modules/lsst.pipe.tasks/tasks/lsst.pipe.tasks.assembleCoadd.AssembleCoaddTask.html

In tutorial-notebooks/05_Intro_to_Source_Detection.ipynb, pipe tasks are used, follow that example.

Similar to NB 05, Jeff managed to use a couple of pipe.tasks for image differencing in https://github.com/rubin-dp0/delegate-contributions-dp01/blob/u/jcarlin/diff_im/diff_im/diff_im_test.ipynb

Yusra shows use of tasks and configs in https://github.com/LSSTScienceCollaborations/StackClubCourse/blob/master/Session02/IntroToDataProductsAndTasks.ipynb

From Yusra's NB, I'm trying to follow "Background Subtraction and Task Configuration" section in use of pipe task.

<br>

In [ ]:
from lsst.pipe.tasks.assembleCoadd import AssembleCoaddTask

In [ ]:
myConfig = AssembleCoaddTask.ConfigClass()

In [ ]:
# call the result year3
myConfig.coaddName = 'year3'

In [ ]:
# yes i only want to do selected visits
myConfig.doSelectVisits = True

In [ ]:
# the list of selected visits
myConfig.connections.selectedVisits = visit_list

What else do I need to specify in the config?

In [ ]:
myConfig

In [ ]:
myTask = AssembleCoaddTask(config=myConfig)

In [ ]:
# myTask?

In [ ]:
# help(myTask.run)

**To Figure Out**

1. The `run` method requires inputs.
I suspect it is insufficient to put the selected visits in the Config.
I need to pass a query or dataIds for the inputs I want to coadd.

2. I'm not sure if I should use `.run` or `.runDataRef`. 
The latter's description is "Assemble a coadd from a set of Warps." which is actually all I want to do.

In [ ]:
# result = myTask.run()

In [ ]:
# result = myTask.runDataRef()

In [ ]:
# result

In [ ]:
# help(assembleCoaddTask)

In [ ]:
del myConfig, myTask

<br>
<br>

### 3.2. Try following Nate's recommended way

follow the steps by K-T in:

https://community.lsst.org/t/figuring-out-how-to-call-the-python-api/6432/11

as recommended by Nate in this post about recreating coadds:

https://community.lsst.org/t/coadding-dp0-calexps-from-different-detectors-for-the-same-deepcoadd/6436/8

Nate Lust also recommended to follow:
 * https://pipelines.lsst.io/v/weekly/modules/lsst.pipe.base/creating-a-pipeline.html
 * https://pipelines.lsst.io/v/weekly/modules/lsst.pipe.base/creating-a-pipelinetask.html


**To Figure Out**:
1. what config do i need to set?
2. where am i even telling it to assemble a coadd from the inputs? how does this pipeline know what to do?

In [ ]:
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline

repo and input collection are already defined

In [ ]:
print(repo)
print(collection)

#### create a collection just for me

https://pipelines.lsst.io/py-api/lsst.daf.butler.Registry.html#lsst.daf.butler.Registry.registerCollection

In [ ]:
output_collection = 'u/MelissaGraham/coadd_recreation_nb'

In [ ]:
# butler.registry.registerCollection(output_collection, type='RUN', doc='made for a tutorial')

already done, so it appears in this list

In [ ]:
# for c in sorted(butler.registry.queryCollections()):
#     print(c)

#### create the data query for my visits and detectors

I learned what this should look like from https://pipelines.lsst.io/getting-started/coaddition.html

In [ ]:
data_query = 'visit in ('+visit_list+')'

In [ ]:
print(data_query)

#### set pipeline config

unsure what is needed or how to figure it out

label, key, and value are not needed? because we can use defaults?

In [ ]:
### Use defaults
# label = 
# key = 
# value = 

#### set up and run

but where am i telling this pipeline just to execute the coadd step?

In [ ]:
butler = SimplePipelineExecutor.prep_butler(repo,
                                            inputs=['2.2i/runs/DP0.2/v23_0_1_rc1/PREOPS-905/pilot_tract4431'],
                                            output='u/MelissaGraham/coadd_recreation_nb')

In [ ]:
pipeline = Pipeline.from_uri('${PIPE_TASKS_DIR}/pipelines/DRP.yaml')

In [ ]:
pipeline.addInstrument('LSSTCam-imSim')

In [ ]:
# pipeline.addConfigOverride(label, key, value)

comment out below, not sure what it's doing yet

In [ ]:
# spe = SimplePipelineExecutor.from_pipeline(pipeline, where=data_query, butler=butler)
# quanta = spe.run(True)